## Day28 - NLP

Code :  "Natural Language Processing in Action: Understanding, Analyzing, and Generating Text With Python"

In [32]:
import numpy as np
import pandas as pd
from nlpia.data.loaders import get_data

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import LatentDirichletAllocation as LDiA

from sklearn.model_selection import train_test_split

sms = get_data('sms-spam')
index = ["sms{}{}".format(i,"!") * j for (i,j) in zip(range(len(sms)),sms.spam)]
np.random.seed(42)

INFO:nlpia.futil:Reading CSV with `read_csv(*('C:\\Users\\Francesco\\anaconda3\\lib\\site-packages\\nlpia\\data\\sms-spam.csv',), **{'nrows': None, 'low_memory': False})`...


In [33]:
counter = CountVectorizer(tokenizer=casual_tokenize)
bow_docs = pd.DataFrame(counter.fit_transform(raw_documents=sms.text)\
                        .toarray(), index=index)
column_nums, terms = zip(*sorted(zip(counter.vocabulary_.values(),\
counter.vocabulary_.keys())))
bow_docs.columns = terms

ldia = LDiA(n_components=16, learning_method='batch')
ldia = ldia.fit(bow_docs)
ldia.components_.shape

components = pd.DataFrame(ldia.components_.T, index=terms)
ldia16_topic_vectors = ldia.transform(bow_docs)
ldia16_topic_vectors = pd.DataFrame(ldia16_topic_vectors, index=index)

In [46]:
X_train, X_test, y_train, y_test = train_test_split(ldia16_topic_vectors, sms.spam, 
                                                    test_size=0.5,random_state=271828)
lda = LDA(n_components=1)
lda = lda.fit(X_train, y_train)
sms['ldia16_spam'] = lda.predict(ldia16_topic_vectors)
round(float(lda.score(X_test, y_test)), 2)

0.94